## Imports

In [ ]:
import keras
import numpy as np
import cv2
import math
from matplotlib import pyplot as plt
from keras_retinanet.utils.anchors import anchors_for_shape, compute_gt_annotations, AnchorParameters
from keras_retinanet.utils.visualization import draw_annotations, draw_caption_debug, draw_boxes, draw_box
from keras_retinanet.utils.colors import label_color

## Define Utility Functions

In [ ]:
def draw_caption2(image, box, caption):
    """ Draws a caption above the box in an image.

    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 5, lineType=cv2.LINE_AA)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3, lineType=cv2.LINE_AA)
    
def draw_caption3(image, box, caption):
    """ Draws a caption above the box in an image.

    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 5, lineType=cv2.LINE_AA)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 5, (255, 0, 0), 3, lineType=cv2.LINE_AA)

## Setup Anchors

In [ ]:
#Default Anchor Settings:
# sizes   = [32, 64, 128, 256, 512]
# strides = [8, 16, 32, 64, 128]
# ratios  = np.array([0.5, 1, 2], keras.backend.floatx())
# scales  = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

In [ ]:
#Customized Anchor Settings:
sizes   = [32, 64, 128, 256, 512]
strides = [8, 16, 32, 64, 128]
ratios  = np.array([0.5, 1, 2], keras.backend.floatx())
scales  = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

In [ ]:
config = [ratios,scales,sizes,strides] #uncomment to setup anchors
#config = None

## Visualize Anchors

In [ ]:
#input image shape
image = np.ones([800, 800, 3])
image = (image*255).astype('uint8')

In [ ]:
ratios,scales,sizes,strides = config
anchor_params = AnchorParameters(sizes, strides, ratios, scales)
anchors = anchors_for_shape(image.shape, anchor_params=anchor_params)

In [ ]:
anchors.shape

In [ ]:
x = 119646+24*9
#draw_boxes(image, anchors[x:x+9], (255, 255, 255), thickness=3)

for idx,box in enumerate(anchors[x:x+9]):
    color = label_color(idx)
    b = box.astype(int)
    draw_box(image, b, color=color, thickness=3)
    draw_caption2(image, b, str(idx+1), )
    
plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(image)
plt.show()

## Version 2.0

In [ ]:
offset = 600
height = image.shape[0]
width = image.shape[1]

In [ ]:
image2 = np.ones([2000, 2000, 3])
image2 = (image2*255).astype('uint8')
draw_box(image2, [offset,offset,offset+width-1,offset+height-1], color=[0,0,0], thickness=-1)

for idx,box in enumerate(anchors[x:x+9]):
    color = label_color((idx)//3+10)
    b = box.astype(int)
    b = b+offset
    draw_box(image2, b, color=color, thickness=3)
    
for idx,box in enumerate(anchors[x:x+9]):
    b = box.astype(int)
    b = b+offset
    draw_caption3(image2, b, str(idx+1), )

plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(image2)
plt.show()

## Version 3.0

In [ ]:
pyramid_levels = [3,4,5,6,7]
num_anchors_at_loc = len(ratios)*len(scales)
num_locs_at_level = []
num_anchors_at_level = []

for i in range(len(pyramid_levels)):
    level = pyramid_levels[i]
    stride = 2**level
    num_locs_at_level.append(math.floor((width+stride-1)/stride)*(math.floor((height+stride-1)/stride)))
    num_anchors_at_level.append(num_locs_at_level[i]*num_anchors_at_loc)
    #print(num_locs_at_level[i])
    #print(num_anchors_at_level[i])

In [ ]:
num_locs_at_level

In [ ]:
num_anchors_at_level

In [ ]:
for idx in range(5):
    plot_level = idx+1
    print('Pyramid level {}'.format(plot_level))
    y = sum(num_anchors_at_level[:plot_level-1])+(num_locs_at_level[plot_level-1]//2)*num_anchors_at_loc
    print('{} anchors'.format(num_anchors_at_level[idx]))
    
    offset = 600
    height = image.shape[0]
    width = image.shape[1]

    image3 = np.ones([2000, 2000, 3])
    image3 = (image3*255).astype('uint8')
    draw_box(image3, [offset,offset,offset+width-1,offset+height-1], color=[0,0,0], thickness=-1)

    for idx,box in enumerate(anchors[y:y+num_anchors_at_loc]):
        color = label_color((idx)//3+10)
        b = box.astype(int)
        b = b+offset
        draw_box(image3, b, color=color, thickness=3)

    for idx,box in enumerate(anchors[y:y+num_anchors_at_loc]):
        b = box.astype(int)
        b = b+offset
        draw_caption3(image3, b, str(idx+1), )

    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image3)
    plt.show()

## End